# Test Multimodal - Extraction de ticket de caisse

Ce notebook permet de tester l'extraction de produits depuis une image de ticket de caisse en utilisant Llama Stack depuis Jupyter sur OpenShift.

## Configuration de l'environnement

## Vérification des services disponibles

Avant de lancer le test, vérifions quels services sont disponibles dans le cluster.

In [ ]:
# Vérifier les services disponibles
import subprocess
import json

def check_service(name, namespace="llama-serve"):
    try:
        result = subprocess.run(
            ["oc", "get", "inferenceservice", name, "-n", namespace, "-o", "json"],
            capture_output=True,
            text=True,
            timeout=10
        )
        if result.returncode == 0:
            data = json.loads(result.stdout)
            url = data.get("status", {}).get("url", "N/A")
            ready = data.get("status", {}).get("conditions", [{}])[0].get("status", "Unknown")
            return {"available": True, "url": url, "ready": ready}
    except:
        pass
    return {"available": False}

print("🔍 Vérification des services disponibles:\n")

# Vérifier llama3-2-3b
llama3_status = check_service("llama3-2-3b")
if llama3_status["available"]:
    print(f"✅ llama3-2-3b disponible: {llama3_status.get('url', 'N/A')}")
    print(f"   Utilisez: export LLAMA_STACK_URL=\"{llama3_status.get('url', '').replace('https://', 'http://')}\"")
else:
    print("❌ llama3-2-3b non disponible")
    print("💡 Déployez-le avec:")
    print("   helm install llama3-2-3b ./helm/03-ai-services/llama3.2-3b -n llama-serve")

# Vérifier llama-stack-instance
try:
    result = subprocess.run(
        ["oc", "get", "pods", "-n", "llama-serve", "-l", "app.kubernetes.io/name=llama-stack-instance", "-o", "jsonpath={.items[0].status.phase}"],
        capture_output=True,
        text=True,
        timeout=10
    )
    if result.returncode == 0 and result.stdout.strip() == "Running":
        print("\n✅ llama-stack-instance en cours d'exécution")
    else:
        print("\n❌ llama-stack-instance non disponible ou en erreur")
        print("💡 Utilisez directement vLLM si disponible")
except:
    print("\n⚠️  Impossible de vérifier llama-stack-instance")

print("\n" + "="*60)

In [ ]:
import os
import sys

# Configuration pour OpenShift
# Depuis Jupyter, on peut accéder directement aux services internes
os.environ.setdefault(
    "LLAMA_STACK_URL",
    "http://llama-stack-instance-service.llama-serve.svc.cluster.local:8321"
)

os.environ.setdefault(
    "MODEL_NAME",
    "meta-llama/Llama-3.2-3B-Instruct"
)

os.environ.setdefault(
    "OTEL_TRACE_ENDPOINT",
    "http://otel-collector-collector.observability-hub.svc.cluster.local:4318/v1/traces"
)

print(f"✅ Configuration:")
print(f"   LLAMA_STACK_URL: {os.environ.get('LLAMA_STACK_URL')}")
print(f"   MODEL_NAME: {os.environ.get('MODEL_NAME')}")
print(f"   OTEL_TRACE_ENDPOINT: {os.environ.get('OTEL_TRACE_ENDPOINT')}")

## Installation des dépendances

In [ ]:
# Installer les dépendances si nécessaire
import subprocess
import sys

def install_package(package):
    try:
        __import__(package.replace('-', '_'))
        print(f"✅ {package} déjà installé")
    except ImportError:
        print(f"📦 Installation de {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} installé")

# Installer les dépendances nécessaires
install_package("requests")
install_package("opentelemetry-api")
install_package("opentelemetry-sdk")
install_package("opentelemetry-exporter-otlp-proto-http")

## Import du script de test

In [ ]:
# Ajouter le répertoire des tests au path si nécessaire
if '.' not in sys.path:
    sys.path.insert(0, '.')

# Importer les fonctions nécessaires
from test_multimodal_receipt import extract_products_from_receipt, setup_tracing

print("✅ Script importé avec succès")

## Configuration du tracing

In [ ]:
# Initialiser le tracing OpenTelemetry
tracer = setup_tracing()
print("✅ Tracing OpenTelemetry configuré")

## Test avec une image de ticket de caisse

In [ ]:
# Chemin vers l'image (ajustez selon votre environnement)
image_path = "download/receipt.jpeg"  # ou le chemin complet vers votre image

# Vérifier que l'image existe
import os
if not os.path.exists(image_path):
    print(f"❌ L'image {image_path} n'existe pas")
    print("💡 Vous pouvez:")
    print("   1. Télécharger une image avec download_receipt_image.py")
    print("   2. Uploader une image via l'interface Jupyter")
    print("   3. Utiliser une image existante dans votre workspace")
else:
    print(f"✅ Image trouvée: {image_path}")
    print(f"   Taille: {os.path.getsize(image_path)} bytes")

## Extraction des produits

In [ ]:
# Extraire les produits du ticket de caisse
if os.path.exists(image_path):
    try:
        print(f"🔍 Analyse du ticket de caisse: {image_path}")
        print(f"📡 Connexion à Llama Stack: {os.environ.get('LLAMA_STACK_URL')}")
        print(f"🤖 Modèle: {os.environ.get('MODEL_NAME')}\n")
        
        result = extract_products_from_receipt(image_path)
        
        # Afficher les résultats
        print("=" * 60)
        print("📋 RÉSULTATS DE L'EXTRACTION")
        print("=" * 60)
        
        if "store" in result and result["store"]:
            print(f"🏪 Magasin: {result['store']}")
        
        if "date" in result and result["date"]:
            print(f"📅 Date: {result['date']}")
        
        print(f"\n🛒 Produits ({len(result.get('products', []))}):")
        print("-" * 60)
        
        total_calculated = 0.0
        for i, product in enumerate(result.get("products", []), 1):
            name = product.get("name", "N/A")
            price = product.get("price", 0.0)
            quantity = product.get("quantity", 1)
            
            print(f"{i}. {name}")
            print(f"   Prix: {price:.2f} €")
            if quantity > 1:
                print(f"   Quantité: {quantity}")
            
            total_calculated += price * quantity
            print()
        
        print("-" * 60)
        if result.get("total"):
            print(f"💰 Total (extrait): {result['total']} €")
        print(f"💰 Total (calculé): {total_calculated:.2f} €")
        print("=" * 60)
        
        # Sauvegarder les résultats
        import json
        output_file = "receipt_extraction_result.json"
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(result, f, indent=2, ensure_ascii=False)
        
        print(f"\n💾 Résultats sauvegardés dans: {output_file}")
        
    except Exception as e:
        print(f"\n❌ Erreur: {e}")
        import traceback
        traceback.print_exc()

## Affichage des résultats en format JSON

In [ ]:
# Afficher les résultats en format JSON structuré
if os.path.exists("receipt_extraction_result.json"):
    import json
    with open("receipt_extraction_result.json", "r", encoding="utf-8") as f:
        result = json.load(f)
    
    print("📊 Résultats JSON:")
    print(json.dumps(result, indent=2, ensure_ascii=False))